In [ ]:
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from keras.layers import Dropout
import os
import math
import random
import numpy as np
import cv2
import pickle

In [ ]:
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train / 255.0
	test_norm = test / 255.0
	# return normalized images
	return train_norm, test_norm


In [ ]:
# def load_dataset():
#     imgs_path = "/content/drive/MyDrive/CIFAR10/"
#     classes = sorted(os.listdir(imgs_path))[1:]

#     n_classes = 4
#     y_onehot = []

#     for i in range(n_classes):
#         alist = []
#         for j in range(n_classes):
#             if i==j:
#                 alist.append(1)
#             else:
#                 alist.append(0)
#         y_onehot.append(alist)


#     X_train = []
#     y_train = []
#     X_test = []
#     y_test = []


#     for i in range(n_classes):
#         class1 = classes[i]
#         imgs = sorted(os.listdir(imgs_path + class1))
#         train_imgs = imgs[:80]
#         test_imgs = imgs[80:]
#         print("train: " + str(len(train_imgs)) + "test: " + str(len(test_imgs)))

#         for j in range(len(train_imgs)):
#             X_train.append(cv2.imread(imgs_path + class1 + '/' + train_imgs[j]))
#             y_train.append(y_onehot[i])

#         for j in range(len(test_imgs)):
#             X_test.append(cv2.imread(imgs_path + class1 + '/' + test_imgs[j]))
#             y_test.append(y_onehot[i])

#     X_train = np.array(X_train).astype('float32')
#     X_test = np.array(X_test).astype('float32')
#     y_train = np.array(y_train).astype('float32')
#     y_test = np.array(y_test).astype('float32')

    
#     X_train, X_test = prep_pixels(X_train, X_test)

#     return X_train, y_train, X_test, y_test

In [ ]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
 
	trainX, testX = prep_pixels(trainX, testX)

	return trainX, trainY, testX, testY

In [ ]:
# def summarize_diagnostics(history):
# 	# plot loss
# 	pyplot.subplot(211)
# 	pyplot.title('Cross Entropy Loss')
# 	pyplot.plot(history.history['loss'], color='blue', label='train')
# 	pyplot.plot(history.history['val_loss'], color='orange', label='test')
# 	# plot accuracy
# 	pyplot.subplot(212)
# 	pyplot.title('Classification Accuracy')
# 	pyplot.plot(history.history['accuracy'], color='blue', label='train')
# 	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
# 	# save plot to file
# 	filename = sys.argv[0].split('/')[-1]
# 	pyplot.savefig(filename + '_plot.png')
# 	pyplot.close()


In [ ]:
def return_choice_init(val1,val2,log):
    if log:
        val1 = math.log(val1,3)
        val2 = math.log(val2,3)

        val1 = (val2-val1)*0.33 + val1
        val2 = (val2-val1)*0.67 + val1

        return 3**val1, 3**val2

    return (val2-val1)*0.33 + val1, (val2-val1)*0.67 + val1

In [ ]:
def initialise_search(record):

    init_vals_1 = {}

    init_vals_1["dr"] = [] 
    init_vals_1["kr"] = [] 

    init_vals_2 = {}

    init_vals_2["dr"] = [] 
    init_vals_2["kr"] = [] 

    for i in range(4):
        v1, v2 = return_choice_init(dr_range_init[0],dr_range_init[1],log=True)
        init_vals_1["dr"].append(v1)
        init_vals_2["dr"].append(v2)

    for i in range(7):
        v1, v2 = return_choice_init(kr_range_init[0],kr_range_init[1],log=True)
        init_vals_1["kr"].append(v1)
        init_vals_2["kr"].append(v2)

    init_vals_1["learn"], init_vals_2["learn"] = return_choice_init(learn_range_init[0],learn_range_init[1],log=True)

    init_vals_1["momentum"], init_vals_2["momentum"] = return_choice_init(momentum_range_init[0],momentum_range_init[1],log=True)

    x, y = return_choice_init(epochs_range_init[0],epochs_range_init[1],log=False)
    init_vals_1["epochs"], init_vals_2["epochs"] = int(x), int(y)

    print(init_vals_1["learn"])
    model1, history = build_train_model(init_vals_1)
    init_vals_1["loss"], acc1 = model1.evaluate(testX, testY, verbose=0)
    print("Validation loss of first initialisation: " + str(init_vals_1["loss"]))

    model2, history = build_train_model(init_vals_2)
    init_vals_2["loss"], acc2 = model2.evaluate(testX, testY, verbose=0)
    print("Validation loss of second initialisation: " + str(init_vals_2["loss"]))


    if init_vals_1["loss"] < init_vals_2["loss"]:
        record["loss"].append(init_vals_1["loss"])
        record["acc"].append(acc1)
        return init_vals_1, init_vals_2, model1, record
    
    else:
        record["loss"].append(init_vals_2["loss"])
        record["acc"].append(acc2)
        return init_vals_2, init_vals_1, model2, record

In [ ]:
def return_divisions(min,max,n_divs):
    arr = []

    diff = (max-min)/n_divs

    curr_num = min+(diff/2)
    arr.append(curr_num)
    for i in range(n_divs-1):
        curr_num = curr_num + diff
        arr.append(curr_num)

    return arr

In [ ]:
def return_choice(val1,val2,loss1,loss2,log):

    min = None
    max = None
    minloss = None
    maxloss = None

    if val1<val2:
        min = val1
        max = val2
        minloss=loss1
        maxloss=loss2
    else:
        min = val2
        max = val1
        minloss=loss2
        maxloss=loss1

    maxrat = int((minloss/(minloss+maxloss))*100)
    minrat = int((maxloss/(minloss+maxloss))*100)

    if minrat==0:
        minrat = 1
        maxrat = 99
    
    if maxrat==0:
        minrat=99
        maxrat=1

    if log:
        min = math.log(min,3)
        max = math.log(max,3)


    mid = (min+max)/2


    min = min - (mid/2)
    max = max + (mid/2)

    arr1 = return_divisions(min,mid,minrat)
    arr2 = return_divisions(mid,max,maxrat)

    choice_range = arr1+arr2

    choice = random.choice(choice_range)

    if log:
        choice = 3**choice

    return choice

In [ ]:
def return_new_vals(best_1,best_2):
    new = {}

    new["dr"] = [] 
    new["kr"] = [] 

    for i in range(4):
        new["dr"].append(return_choice(best_1["dr"][i],best_2["dr"][i],best_1["loss"],best_2["loss"],log=True))

    for i in range(7):
        new["kr"].append(return_choice(best_1["kr"][i],best_2["kr"][i],best_1["loss"],best_2["loss"],log=True))

    new["learn"] = return_choice(best_1["learn"],best_2["learn"],best_1["loss"],best_2["loss"],log=True)

    new["momentum"] = return_choice(best_1["momentum"],best_2["momentum"],best_1["loss"],best_2["loss"],log=True)

    new["epochs"] = int(return_choice(best_1["epochs"],best_2["epochs"],best_1["loss"],best_2["loss"],log=False))

    return new;

In [ ]:
def build_train_model(var_dict):
    print(var_dict)
    dr_i, kr_i, learn_i, momentum_i, epoch_i = var_dict["dr"], var_dict["kr"], var_dict["learn"], var_dict["momentum"], var_dict["epochs"]

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(kr_i[0]), input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(kr_i[1])))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dr_i[0]))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(kr_i[2])))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(kr_i[3])))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dr_i[1]))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(kr_i[4])))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(kr_i[5])))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dr_i[2]))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(kr_i[6])))
    model.add(Dropout(dr_i[3]))
    model.add(Dense(10, activation='softmax'))
    # compile model
    opt = SGD(lr=learn_i, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(trainX, trainY, epochs=10, batch_size=64, validation_data=(testX, testY), verbose=1)

    return model, history

In [ ]:
def update_best(best_1,best_2,new, model):
    if new["loss"] < best_1["loss"]:
        model.save('/content/drive/MyDrive/Weighted_Random')
        return new, best_1

    elif new["loss"] < best_2["loss"]:
        return best_1, new

    else:
        return best_1, best_2

In [ ]:
# load dataset
# prepare pixel data
# define model
trainX, trainY, testX, testY = load_dataset()
dr_range_init = [0.01,0.5]
kr_range_init = [0.01,0.1]
learn_range_init = [0.001,0.01]
momentum_range_init = [0.99,0.999]
epochs_range_init = [5,25]

total_iter = 5

In [ ]:
record = {}
record["loss"] = []
record["acc"] = []

best_1, best_2, best_model, record = initialise_search(record)
record_file = open('/content/drive/MyDrive/Weighted_Random/record.pkl', "wb")
pickle.dump(record, record_file)
best_model.save('/content/drive/MyDrive/Weighted_Random/')

In [ ]:
for i in range(total_iter):
    new = return_new_vals(best_1,best_2)

    model, history = build_train_model(new)

    new["loss"], acc = model.evaluate(testX, testY, verbose=0)
    record["loss"].append(new["loss"])
    record["acc"].append(acc)

    best_1, best_2 = update_best(best_1,best_2,new, model)

    record_file = open('/content/drive/MyDrive/Weighted_Random/record.pkl', "wb")
    pickle.dump(record, record_file)
    best_model.save('/content/drive/MyDrive/Weighted_Random/')
    print("Validation Loss of this Iteration: " + str(new["loss"]))
    print("Validation Loss of Best Model: " + str(best_1["loss"]))